In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re
from urllib.request import urlopen

In [ ]:
start = time.time()
time.sleep(5)
end = time.time()
end - start

# Scraping all the threads in the forum

In [5]:
start = time.time()

title_list = []
url_list = []
replies_list = []
views_list = []


for i in range (1,1250):
    temp_url = 'https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/index' + str(i) + '.html'
    print (temp_url)
    temp_response = requests.get(temp_url)
    temp_soup = BeautifulSoup(temp_response.text.replace('ISO-8859-1', 'utf-8'), 'lxml') #lxml parser allows both html and xml to be parsed
    temp_tbody = temp_soup.find_all('tbody')
    temp_thread_tbody = temp_tbody[4]
    temp_thread_tbody_list = temp_thread_tbody.find_all('td', id=lambda x: x and x.startswith('td_threadtitle_'))
    temp_thread_tbody_repliesviews_list = temp_thread_tbody.find_all('td', title=lambda x: x and x.startswith('Replies'))

      
    for num in range(len(temp_thread_tbody_list)):
        try:
            temp_body = temp_thread_tbody_list[num]
            temp_title = temp_body.find_all('a')[0].text
            temp_spec_url = temp_body.find_all('a')[0]['href']

          # append title into title list
            title_list.append(temp_title)
          # append url into url list
            url_list.append(temp_spec_url)
        except:
            title_list.append(None)
            url_list.append(None)     

        try:
          # get the replies and views
            temp_repliesviews = temp_thread_tbody_repliesviews_list[num] #for num in range ...
            temp_stuff = temp_repliesviews['title']     
          # get reply count
            replies_list.append(temp_stuff.split(', ')[0].split(': ')[-1])
        # get view count
            views_list.append(temp_stuff.split(', ')[1].split(': ')[-1].replace(",", ""))
        except:
            replies_list.append(None)
            views_list.append(None)
    time.sleep(1.5)
    
example_df = pd.DataFrame({"title": title_list,
                               "url": url_list,
                               "replies": replies_list,
                               "views": views_list})
end = time.time()
print(end-start) #show example_df at the end

example_df
#example_response


https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/index1.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/index2.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/index3.html


KeyboardInterrupt: 

In [ ]:
example_df.to_csv('scraped_data_again.csv', index = False)

In [3]:
url_edited_df = pd.read_csv('scraped_data_again2.csv')

In [4]:
from urllib.request import urlopen

#url_edited_df = pd.read_csv('scraped_data_again2.csv')
for url in url_edited_df[:10]['url_full']:
    print(url)

https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-noting%5D-chin-swee-road-case-reminder-gag-order-6116293.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/what-you-should-know-about-online-vigilantism-doxxing-what-means-you-6021758.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bpsa%5D-please-post-responsibly-avoid-misleading-others-6191463.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/reporting-execution-bumper-clones-all-disturbing-photo-posters-%5Btemporarily-closed%5D-6201994.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%2A%2A%5Bimportant%5D-edmw-must-reads-%2A%2A-2005501.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/advanced-happy-new-year-all-some-important-notes-5271881.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/rant-online-food-ordering-6282010.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-polic

## Checking

In [5]:
url_to_follow = url_edited_df.iloc[0]['url_full']
url_to_follow

'https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-noting%5D-chin-swee-road-case-reminder-gag-order-6116293.html'

# Scraping posts within each thread

In [19]:
'''
pages = bs.find('div', class_ = "pagination")
span = pages.span
no_of_pages = int(span.text.strip("Page 1 of"))

This function works OK and it can go into depth of any thread and output replieas from ALL pages!
'''


def follow_one_post(url_to_follow):
    msg_response = requests.get(url_to_follow)
    thread_count = 0
    comment_count = 0
    thread_title_list = []
    comment_author_list = []
    date_list = []
    comment_list = []
    soup = BeautifulSoup(msg_response.text.replace('ISO-8859-1', 'utf-8'), 'html.parser')
    ###
    try:
        pages = soup.find('div', class_ = "pagination")
        span = pages.span
        no_of_pages = int(span.text.strip("Page 1 of"))
    except:
        no_of_pages = 1
    ###
    base_url = url_to_follow.replace('.html', '')
    
    for page_n in range(1,no_of_pages+1):
        #print(page_n, 'of', no_of_pages)
    ## insert the code to iterate all pages, adding page number at the end of url
    ## [official]-tpg-telecom-part-2-a-5980745-3.html
    ## [official]-tpg-telecom-part-2-a-5980745.html
        if page_n > 1:
            work_url = base_url+'-'+str(page_n)+'.html'
        else:
            work_url = base_url+'.html'
        print(page_n, 'of', no_of_pages, work_url)
        msg_response = requests.get(work_url)
        soup = BeautifulSoup(msg_response.text.replace('ISO-8859-1', 'utf-8'), 'html.parser')
        replies = soup.find_all('div', class_ ='post-wrapper')
        thread = soup.find('h2', {'class':'header-gray'})
        thread_title = thread.text
        thread_count += 1
        for container in replies:
                date = container.find('td').text.strip('\n\t\r ')
                #if '2018' not in date and '2019' not in date:
                #    continue
                #else:
                #print(date)
                comment_count+=1
                #Capture HTML comment section 
                comment = container.find('div',attrs = {'class':'post_message'})
                if comment:
                    comment = comment.text
                else:
                    comment = '' # Comments that have no text (eg. only pictures)
                if container.find('div',{'class':'quote'}): # If comment contains quote,
                    for quote in container.find_all('div',{'class':'quote'}):
                        comment = comment.replace(quote.text, '') # Remove quote


                #Capture HTML author section
                comment_author = container.find('a',{'class':'bigusername'})
                if comment_author:
                    comment_author = comment_author.text
                else: # guest users will not have a 'bigusername' class, so just call all guest users 'EDMW Guest User'
                    comment_author = 'EDMW Guest User'

                thread_title_list.append(thread_title.strip())           
                comment_author_list.append(comment_author)
                date_list.append(date)
                comment_list.append(comment)
                
    # you'll have to add this line to return the df    
    result_df = pd.DataFrame({"thread_title": thread_title_list,
                            "comment_author": comment_author_list,
                            "date": date_list,
                            "comment": comment_list})
    # return the df                       
    return result_df

In [20]:
start = time.time()
url_to_follow = url_edited_df.iloc[7]['url_full']
result = follow_one_post(url_to_follow)

print(time.time() - start)

1 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161.html
2 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161-2.html
3 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161-3.html
4 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161-4.html
5 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161-5.html
6 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-85-year-old-man-caught-vice-related-raid-police-6282161-6.html
7.091562509536743


In [21]:
result

,thread_title,comment_author,date,comment
0,"30 women arrested for suspected vice activity,...",s-ghost,"10-05-2020, 10:19 PM","30 women arrested for suspected vice activity,..."
1,"30 women arrested for suspected vice activity,...",Tough times ahead,"10-05-2020, 10:19 PM",Anti vice raids during CB\n\nhttps://www.8worl...
2,"30 women arrested for suspected vice activity,...",Tough times ahead,"10-05-2020, 10:20 PM",85 still can shoot?
3,"30 women arrested for suspected vice activity,...",BlackTortoise,"10-05-2020, 10:23 PM",How the freelance chicken get into Singapore? ...
4,"30 women arrested for suspected vice activity,...",makann,"10-05-2020, 10:23 PM","A total of 30 women, aged between 19 and 48,\n..."
5,"30 women arrested for suspected vice activity,...",m0n0n0ke,"10-05-2020, 10:24 PM",85? I salute u
6,"30 women arrested for suspected vice activity,...",whatmeworry,"10-05-2020, 10:25 PM",Really Suay to get caught.
7,"30 women arrested for suspected vice activity,...",treeskull,"10-05-2020, 10:25 PM",Wtf 85\n\nSent from 你是我的小啊小苹果儿怎么爱你都不嫌多红红的小脸儿温暖...
8,"30 women arrested for suspected vice activity,...",maxieb,"10-05-2020, 10:26 PM",85 ah pek call cheekon more satki
9,"30 women arrested for suspected vice activity,...",Idme231,"10-05-2020, 10:26 PM",We must be one of the most active crimes happe...


In [17]:
from concurrent.futures import ThreadPoolExecutor, as_completed

with ThreadPoolExecutor(max_workers=5) as executor:
    start = time.time()
    df_list = []
    future_to_url = [executor.submit(follow_one_post, url) for url in url_edited_df[:150]['url_full']]
    for result in as_completed(future_to_url):
        df_list.append(result.result())
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))    

1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/reporting-execution-bumper-clones-all-disturbing-photo-posters-%5Btemporarily-closed%5D-6201994.html
1 of 2 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-noting%5D-chin-swee-road-case-reminder-gag-order-6116293.html
1 of 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/what-you-should-know-about-online-vigilantism-doxxing-what-means-you-6021758.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%2A%2A%5Bimportant%5D-edmw-must-reads-%2A%2A-2005501.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bpsa%5D-please-post-responsibly-avoid-misleading-others-6191463.html
2 of 2 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-noting%5D-chin-swee-road-case-reminder-gag-order-6116293-2.html
2 of 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/what-you-should-know-about-online-vigilantism-doxxing-what-means-you-6021758-2.html
1 of 7 https://

3 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-continues-part-98-a-6253105-3.html
13 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-54-a-6281684-13.html
6 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/me-my-buddys-gf-michelle-6281757-6.html
7 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-7.html
4 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-continues-part-98-a-6253105-4.html
1 of 9 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-ceca-creates-thread-sg-subreddit-cries-about-racism-6281561.html
14 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-54-a-6281684-14.html
28 of 628 https://forums.hardwarezone.com.sg/eat

24 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-54-a-6281684-24.html
18 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-18.html
3 of 536 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-tpg-telecom-part-2-a-5980745-3.html
16 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-continues-part-98-a-6253105-16.html
19 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-19.html
11 of 82 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgvgt%5D%5Bsaga%5D-gang-clash-3am-knife-motor-helmet-still-bring-gf-come-6281823-11.html
25 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-pa

29 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-29.html
36 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-54-a-6281684-36.html
23 of 82 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgvgt%5D%5Bsaga%5D-gang-clash-3am-knife-motor-helmet-still-bring-gf-come-6281823-23.html
27 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-continues-part-98-a-6253105-27.html
14 of 536 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-tpg-telecom-part-2-a-5980745-14.html
30 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-30.html
37 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-

4038 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-continues-part-98-a-6253105-38.html of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-40.html

26 of 536 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-tpg-telecom-part-2-a-5980745-26.html
41 of 628 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bemergency-preparedness-thread%5D-how-pack-emergency-bag-related-discussions-sg-context-5061529-41.html
48 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-54-a-6281684-48.html
35 of 82 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgvgt%5D%5Bsaga%5D-gang-clash-3am-knife-motor-helmet-still-bring-gf-come-6281823-35.html
39 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/lai-liao-lai-liao-regina-kam-saga-con

...

429 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-429.html
597 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-597.html
430 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-430.html
598 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-598.html
431 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-431.html
599 of 432 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-599.html
667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-432.html
600 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-wil

628 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-628.html
460 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-460.html
629 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-629.html
461 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-461.html
630 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-630.html
462 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-462.html
631 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-631.html
463 of 667 https://forums.hardwarezone.com

659 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-659.html
492 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-492.html
660 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-660.html
493 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-493.html
661 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-661.html
494 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-494.html
662 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5B-%5D-how-many-will-protest-sunday-%5Blive-stream%5D%5B-%5D-part-8-a-6150978-662.html
495 of 667 https://forums.hardwarezone.com

554 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-554.html
555 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-555.html
556 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-556.html
557 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-557.html
558 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-558.html
559 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-559.html
560 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-560.html
561 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-561.html
562 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-562.html
563 of 667

629 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-629.html
630 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-630.html
631 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-631.html
632 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-632.html
633 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-633.html
634 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-634.html
635 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-635.html
636 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-636.html
637 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i-am-part-8-a-6265345-637.html
638 of 667

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from requests.exceptions import Timeout
from requests.exceptions import HTTPError

with ThreadPoolExecutor(max_workers=5) as executor:
    start = time.time()
    df_list_2 = []
    future_to_url_2 = [executor.submit(follow_one_post, url) for url in url_edited_df[150:600]['url_full']]
    for result in as_completed(future_to_url_2):
        try:
            response = requests.get(url)
            df_list_2.append(result.result())
            response.raise_for_status()
        except HTTPError as http_err:
            pass
        except Exception as err:
            pass
    
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))   

In [37]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from requests.exceptions import Timeout
from requests.exceptions import HTTPError

with ThreadPoolExecutor(max_workers=5) as executor:
    start = time.time()
    df_list_3 = []
    future_to_url_3 = [executor.submit(follow_one_post, url) for url in url_edited_df[600:800]['url_full']]
    for result in as_completed(future_to_url_3):
        try:
            response = requests.get(url)
            df_list_3.append(result.result())
            response.raise_for_status()
        except HTTPError as http_err:
            pass
        except Exception as err:
            pass
    
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))   

1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/share-your-bad-working-experience-mnc-6281800.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/everyday-life-bygone-days-tokyo-1966-a-6281801.html
1 of 2 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/should-pap-just-write-off-200k-fws-6260164.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/thank-you-pap-opening-up-mcdonalds-again-today-also-swee-3-cases-only-6281798.html
1 of 5 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/guess-mrt-stations-6281739.html
2 of 22 of 5 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/guess-mrt-stations-6281739-2.html
 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/should-pap-just-write-off-200k-fws-6260164-2.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/we-expected-wear-mask-daily-until-vaccine-corvid-found-6281751.html
1 of 7 https://forums.hardwarezone.com.sg/eat-drink-man-woman-

2 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/heavy-rain-6281427-2.html
14 of 22 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anibodie-wan-pray-animal-crossing-tgt-6269644-14.html
3 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgpgt%5D-anyone-rike-plain-yogurt-sinkie-made-de-6281511-3.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/eating-habits-during-covid-19-a-6281755.html
15 of 22 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anibodie-wan-pray-animal-crossing-tgt-6269644-15.html
1 of3 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/heavy-rain-6281427-3.html
 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/how-malaysia-lockdown-6265320.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/alvin-tan-aka-boeing767-6281771.html
2 of 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/how-malaysia-lockdown-6265320-2.html
1 of 1 https://forums.hardwarez

1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/how-can-call-tech-come-6281746.html
7 of 7 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/serious-why-malaysia-1-million-fw-got-no-covid-spread-6279180-7.html
3 11 of 36 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sadisticnoob-beginner-unit-trust-etf-snowball-portfolio-5830348-11.html
of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/coolermaster-aio-cpu-cooler-good-6280975-3.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/offical-expecting-speech-pm-lhl-1-2-wks-time-6281745.html
12 of 36 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sadisticnoob-beginner-unit-trust-etf-snowball-portfolio-5830348-12.html
1 of 6 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/former-singapore-embassy-staff-gets-114-months-jail-6281584.html
1 of 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/movie-chow-yun-fatt-best-6281512.html
1 of 2 https://for

257 of 36 of 24  https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sadisticnoob-beginner-unit-trust-etf-snowball-portfolio-5830348-25.html
https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/trump-administration-%93turbocharging%94-initiative-remove-global-supply-chains-china-6277353-7.html
1 of 34 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/gpgt-show-your-xiasuay-stockpile-6240127.html
9 of 32 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%24200-left-sam-his-five-family-members-kana-sexposed-not-telling-full-story-6268790-9.html
2 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/indon-youtuber-faces-12-years-jail-prank-video-giving-garbage-filled-donations-6281574-2.html
10 of 32 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%24200-left-sam-his-five-family-members-kana-sexposed-not-telling-full-story-6268790-10.html
3 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/indon-youtuber-faces-12-years-jail-prank

22 of 32 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%24200-left-sam-his-five-family-members-kana-sexposed-not-telling-full-story-6268790-22.html
14 of 2034 of 24 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/trump-administration-%93turbocharging%94-initiative-remove-global-supply-chains-china-6277353-20.html
 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/gpgt-show-your-xiasuay-stockpile-6240127-14.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgpgt-saga%5D-edmw%92er-kena-trash-sinkie-xmm-6281471.html
1 of 412 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/master-lus-world-tours-singapore-may-2018-a-5763017.html
23 of 32 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%24200-left-sam-his-five-family-members-kana-sexposed-not-telling-full-story-6268790-23.html
15 of 34 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/gpgt-show-your-xiasuay-stockpile-6240127-15.html
21 of 24 https://forums.hardw

...

332 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-332.html
521 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-521.html
333 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-333.html
522 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-522.html
334 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-334.html
523 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-523.html
335 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-335.html
524 of 667 https:

362 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-362.html
550 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-550.html
363 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-363.html
551 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-551.html
364 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-364.html
552 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-552.html
365 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-365.html
553 of 667 https:

579 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-579.html
392 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-392.html
580 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-580.html
393 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-393.html
581 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-581.html
394 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-394.html
582 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-582.html
395 of 423 https://forums.hard

422 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-422.html
608 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-608.html
423609 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-609.html
 of 423 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anniversary-late-mr-ong-teng-cheong-lets-make-100-000-posts-2263203-423.html
610 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-610.html
611 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-611.html
612 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bnight-owls-den%5D-awake-care-chat-part-53-a-6249965-612.html
613 of 667 https://forums.hardwarezone.com.

Time Taken: 701.676529s


In [44]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from requests.exceptions import Timeout
from requests.exceptions import HTTPError

with ThreadPoolExecutor(max_workers=5) as executor:
    start = time.time()
    df_list_4 = []
    future_to_url_4 = [executor.submit(follow_one_post, url) for url in url_edited_df[800:1250]['url_full']]
    for result in as_completed(future_to_url_4):
        try:
            response = requests.get(url)
            df_list_4.append(result.result())
            response.raise_for_status()
        except HTTPError as http_err:
            pass
        except Exception as err:
            pass
    
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start)) 

1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/who-pilot-byarlant-custom-6281635.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/why-got-women-come-edmw-one-6281643.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/breaking-where-can-buy-n95-mask-6281577.html
1 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/where-mummy1234-tis-crisis-6281495.html
1 of 20 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/today-alot-edmwers-kena-banned-6165794.html
2 of 3 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/where-mummy1234-tis-crisis-6281495-2.html
2 of 20 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/today-alot-edmwers-kena-banned-6165794-2.html
1 of 4 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sia-silkair-suspend-meal-service-flight-within-sea-china-6281521.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/anything-china-does-right-brainwashed-6281557.

5 of 5 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bgpgt%5D-singaporean-guy-breaking-circuit-breaker-laws-flaunting-his-actions-instagram-6281415-5.html
13 of 7 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-mdm-ho-feedback-tracetogether-6281261-3.html
 of 16 of 20 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/today-alot-edmwers-kena-banned-6165794-16.html
1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/ang-kong-bin-tio-hoot-gg-6281609.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/why-some-edmwers-ti-ti-want-cb-extend-6281620.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/any-comments-marble-striploin-6281603.html
4 of 7 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bglgt%5D-mdm-ho-feedback-tracetogether-6281261-4.html
17 of 20 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/today-alot-edmwers-kena-banned-6165794-17.html
1 of 4 https://forums.hardwarezone

3 of 477 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-crypto-currency-chit-chat-group-part-3-a-5772327-3.html
11 of 2  https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sk-13-new-cases-single-person-who-attended-five-nightclubs-bars-6281482.html
of 2 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/what-happened-no-evidence-community-spread-lol-6281464.html
4 of 74 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bask-jwee85%5D-motorcycles-stuff-i-will-try-answer-within-my-knowledge-part-3-a-5872121-4.html
11 of 24 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i%92m-35-unkle-own-small-business-6179955-11.html
5 of 74 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bask-jwee85%5D-motorcycles-stuff-i-will-try-answer-within-my-knowledge-part-3-a-5872121-5.html
2 of 2 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/sk-13-new-cases-single-person-who-attended-five-nightclubs-bars-6281482-2.htm

16 of 477 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-crypto-currency-chit-chat-group-part-3-a-5772327-16.html
17 of 74 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bask-jwee85%5D-motorcycles-stuff-i-will-try-answer-within-my-knowledge-part-3-a-5872121-17.html
24 of 24 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bama%5D-i%92m-35-unkle-own-small-business-6179955-24.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/singapore-oil-trader-zenrock-involved-dishonest-deals-says-hsbc-6280043.html
18 of 74 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bask-jwee85%5D-motorcycles-stuff-i-will-try-answer-within-my-knowledge-part-3-a-5872121-18.html
17 of 477 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-crypto-currency-chit-chat-group-part-3-a-5772327-17.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/wishing-everyone-good-cb-sunday-morning-6281587.html
1 o

1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/last-time-channel-8-got-one-show-similar-incredible-tales-hosted-guo-liang-6281567.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/last-time-u-know-any-soldier-smart-6280842.html
29 of 477 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-crypto-currency-chit-chat-group-part-3-a-5772327-29.html
30 of 74 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bask-jwee85%5D-motorcycles-stuff-i-will-try-answer-within-my-knowledge-part-3-a-5872121-30.html
1 of 23 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/jiuhu-kia-sly-people-who-can-entrap-you-many-ways-6264247.html
1 of 1 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/do-u-rmb-incredible-tales-6281565.html
30 of 477 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%5Bofficial%5D-crypto-currency-chit-chat-group-part-3-a-5772327-30.html
31 of 74 https://forums.hardwarezone.com.sg/eat-drink-ma

...

340 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-340.html
508 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-508.html
341 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-341.html
509 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-509.html
342 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-342.html
510 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-510.html
343 of 413 https://forums.hardwarezone.com.sg/eat-drink-ma

367 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-367.html
535 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-535.html
368 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-368.html
536 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-536.html
369 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-369.html
537 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-537.html
370 of 413 https://forums.hardwarezone.com.sg/eat-drink-ma

561 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-561.html
394 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-394.html
562 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-562.html
395 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-395.html
563 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-563.html
396 of 413 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/taiwanese-models-actresses-hosts-magazine-pictures-part-2-a-4723225-396.html
564 of 667 https://forums.hardwarezone.com.sg/eat-drink-ma

593 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-593.html
594 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-594.html
595 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-595.html
596 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-596.html
597 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-597.html
598 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-5

642 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-642.html
643 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-643.html
644 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-644.html
645 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-645.html
646 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-646.html
647 of 667 https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/%7E%2A-official-barclays-premier-league-betting-section-%2A%7E-sg-pools-part-53-a-6170249-6

In [45]:
test_combined = pd.concat(df_list_4)
from pandas import DataFrame
test_combined_df = DataFrame (test_combined)

test_combined_df

,thread_title,comment_author,date,comment
0,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:01 PM",Who is the pilot for the Byarlant Custom?
1,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:02 PM",Some said it's Kou Uraki
2,Who is the pilot for the Byarlant Custom?,Sausage,"10-05-2020, 12:14 PM",Nina prupleton\n\nSent from Xiaomi REDMI NOTE ...
0,why got women come edmw one,lydiaa,"10-05-2020, 12:13 PM",when got all the btss threads. \n\nPosted from...
1,why got women come edmw one,EveMuffins,"10-05-2020, 12:16 PM",
2,why got women come edmw one,Sammiegirl00,"10-05-2020, 12:16 PM",Nice handle lydiaa ...
3,why got women come edmw one,lydiaa,"10-05-2020, 12:16 PM",moi is fcp \n\nPosted from PCWX using vic
4,why got women come edmw one,huihong,"10-05-2020, 12:17 PM",Cus women also have fetish for bbfa
5,why got women come edmw one,Illusionist_85,"10-05-2020, 12:17 PM",recruit battalion of white knights
0,(Breaking) Where can buy N95 mask ?,Sai Baba,"10-05-2020, 10:49 AM",Indonesia Says Looming Haze Can Worsen Health ...


In [46]:
# start scraping stuff

# in parallel, start collecting the data and analyze

# 10 threads = 7 mins
# 1200 threads = 7 * 120 = 940 minutes

# 5 threads = 5 seconds
# 1200 threads = 5 sec * 240 = 1200 seconds = 20 minutes

# Recommendation:
# Start with 100 first and measure time
# Export first 100

# Continue with 200 and measure time
# Export next 200

# Contact Jackie to decide next strategy

In [47]:
emoji_removed_df = test_combined_df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
emoji_removed_df

,thread_title,comment_author,date,comment
0,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:01 PM",Who is the pilot for the Byarlant Custom?
1,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:02 PM",Some said it's Kou Uraki
2,Who is the pilot for the Byarlant Custom?,Sausage,"10-05-2020, 12:14 PM",Nina prupleton\n\nSent from Xiaomi REDMI NOTE ...
0,why got women come edmw one,lydiaa,"10-05-2020, 12:13 PM",when got all the btss threads. \n\nPosted from...
1,why got women come edmw one,EveMuffins,"10-05-2020, 12:16 PM",
2,why got women come edmw one,Sammiegirl00,"10-05-2020, 12:16 PM",Nice handle lydiaa ...
3,why got women come edmw one,lydiaa,"10-05-2020, 12:16 PM",moi is fcp \n\nPosted from PCWX using vic
4,why got women come edmw one,huihong,"10-05-2020, 12:17 PM",Cus women also have fetish for bbfa
5,why got women come edmw one,Illusionist_85,"10-05-2020, 12:17 PM",recruit battalion of white knights
0,(Breaking) Where can buy N95 mask ?,Sai Baba,"10-05-2020, 10:49 AM",Indonesia Says Looming Haze Can Worsen Health ...


In [48]:
#datacleaning
emoji_removed_df.drop_duplicates(inplace=True)
#Remove white spaces
emoji_removed_df['thread_title'] = emoji_removed_df['thread_title'].str.replace(r'\r|\n|\t','')
emoji_removed_df['comment'] = emoji_removed_df['comment'].str.replace(r'\r|\n|\t',' ')
emoji_removed_df


,thread_title,comment_author,date,comment
0,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:01 PM",Who is the pilot for the Byarlant Custom?
1,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:02 PM",Some said it's Kou Uraki
2,Who is the pilot for the Byarlant Custom?,Sausage,"10-05-2020, 12:14 PM",Nina prupleton Sent from Xiaomi REDMI NOTE 7 ...
0,why got women come edmw one,lydiaa,"10-05-2020, 12:13 PM",when got all the btss threads. Posted from P...
1,why got women come edmw one,EveMuffins,"10-05-2020, 12:16 PM",
2,why got women come edmw one,Sammiegirl00,"10-05-2020, 12:16 PM",Nice handle lydiaa ...
3,why got women come edmw one,lydiaa,"10-05-2020, 12:16 PM",moi is fcp Posted from PCWX using vic
4,why got women come edmw one,huihong,"10-05-2020, 12:17 PM",Cus women also have fetish for bbfa
5,why got women come edmw one,Illusionist_85,"10-05-2020, 12:17 PM",recruit battalion of white knights
0,(Breaking) Where can buy N95 mask ?,Sai Baba,"10-05-2020, 10:49 AM",Indonesia Says Looming Haze Can Worsen Health ...


In [49]:
#Remove GAGT/PCWX app automatic signatures
emoji_removed_df['comment'].replace(r'Sent from .* using GAGT', '', inplace = True, regex=True) 
emoji_removed_df['comment'].replace(r'Posted from PCWX using .*', '', inplace = True, regex=True) 
# ^^ will not remove the entire phone model but as much as possible try 
emoji_removed_df['comment'].replace(r'Posted with .* via PCWX', '', inplace = True, regex=True)
emoji_removed_df

,thread_title,comment_author,date,comment
0,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:01 PM",Who is the pilot for the Byarlant Custom?
1,Who is the pilot for the Byarlant Custom?,Wifi2020,"10-05-2020, 12:02 PM",Some said it's Kou Uraki
2,Who is the pilot for the Byarlant Custom?,Sausage,"10-05-2020, 12:14 PM",Nina prupleton
0,why got women come edmw one,lydiaa,"10-05-2020, 12:13 PM",when got all the btss threads.
1,why got women come edmw one,EveMuffins,"10-05-2020, 12:16 PM",
2,why got women come edmw one,Sammiegirl00,"10-05-2020, 12:16 PM",Nice handle lydiaa ...
3,why got women come edmw one,lydiaa,"10-05-2020, 12:16 PM",moi is fcp
4,why got women come edmw one,huihong,"10-05-2020, 12:17 PM",Cus women also have fetish for bbfa
5,why got women come edmw one,Illusionist_85,"10-05-2020, 12:17 PM",recruit battalion of white knights
0,(Breaking) Where can buy N95 mask ?,Sai Baba,"10-05-2020, 10:49 AM",Indonesia Says Looming Haze Can Worsen Health ...


In [ ]:
    for i in range(len(shop_transactions)-1):   # each time interval
        ts_curr = dt.datetime.strptime(shop_transactions['event_time'].iat[i], '%Y-%m-%d %H:%M:%S')   # converts to datetime object
        list_of_userid = [shop_transactions['userid'].iat[i]]   # stores userid(s) for this time interval
        

In [50]:
emoji_removed_df.to_csv('file3.csv', encoding='utf-8-sig')

In [51]:
list1_df = pd.read_csv('Jackie/first150.csv')
list2_df = pd.read_csv('Jackie/file2_150_600.csv')
list3_df = pd.read_csv('Jackie/file3_600_800.csv')
list4_df = pd.read_csv('Jackie/file4_800_1250.csv')

In [58]:
all_df = pd.concat([list1_df,list2_df,list3_df,list4_df]).reset_index(drop=True)
all_df

,Unnamed: 0,thread_title,comment_author,date,comment
0,0,REPORTING AND EXECUTION OF BUMPER CLONES AND A...,Emperor_Hippo,"04-11-2018, 10:12 AM","Dear all, In view of recent bumper clone atta..."
1,0,**[IMPORTANT] EDMW Must-reads!**,leinad,"23-06-2008, 09:56 PM","1) Discrimination of Race, Culture, Derogatory..."
2,1,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"23-06-2008, 11:50 PM","Dear EDMWers, I would like to remind all of y..."
3,3,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"20-02-2011, 07:28 PM",Amended Infraction Points System and Banning P...
4,4,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"20-02-2011, 09:14 PM",2 points: - Thread Digging or Rubbish/Nuisanc...
5,5,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"30-01-2012, 10:43 AM",Removed sticky thread in EDMW for forum respon...
6,6,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"06-09-2014, 08:22 PM",I've moved a few sticky threads to the feedbac...
7,7,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"31-12-2015, 08:12 PM",Added a link on How to preserve a discussion f...
8,8,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"20-06-2018, 10:44 AM","Removed the sticky thread in EDMW on ""HWZ Acco..."
9,0,[PSA] Please post responsibly to avoid mislead...,Dr.Vijay,"26-01-2020, 08:35 PM",PSA: Don't post important information without ...


In [60]:
import numpy as np
all_df['comment'].replace('', np.nan, inplace = True) #replace empty strings with NaN

all_df

,Unnamed: 0,thread_title,comment_author,date,comment
0,0,REPORTING AND EXECUTION OF BUMPER CLONES AND A...,Emperor_Hippo,"04-11-2018, 10:12 AM","Dear all, In view of recent bumper clone atta..."
1,0,**[IMPORTANT] EDMW Must-reads!**,leinad,"23-06-2008, 09:56 PM","1) Discrimination of Race, Culture, Derogatory..."
2,1,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"23-06-2008, 11:50 PM","Dear EDMWers, I would like to remind all of y..."
3,3,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"20-02-2011, 07:28 PM",Amended Infraction Points System and Banning P...
4,4,**[IMPORTANT] EDMW Must-reads!**,Kiwi8,"20-02-2011, 09:14 PM",2 points: - Thread Digging or Rubbish/Nuisanc...
5,5,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"30-01-2012, 10:43 AM",Removed sticky thread in EDMW for forum respon...
6,6,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"06-09-2014, 08:22 PM",I've moved a few sticky threads to the feedbac...
7,7,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"31-12-2015, 08:12 PM",Added a link on How to preserve a discussion f...
8,8,**[IMPORTANT] EDMW Must-reads!**,Dr.Vijay,"20-06-2018, 10:44 AM","Removed the sticky thread in EDMW on ""HWZ Acco..."
9,0,[PSA] Please post responsibly to avoid mislead...,Dr.Vijay,"26-01-2020, 08:35 PM",PSA: Don't post important information without ...


In [61]:
all_df.to_csv('file_combined.csv', encoding='utf-8-sig')